In [ ]:
from sqlalchemy import create_engine

engine = create_engine(r"sqlite:///C:/Users/blinklet/Documents/chinook-database/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite")

In [ ]:
from sqlalchemy import inspect

inspector = inspect(engine)

print(inspector.get_table_names())

In [ ]:
from pprint import pprint

# Columns in table "Album"
pprint(inspector.get_columns("Album"))
print()

# Primary Key for table "Album"
pprint(inspector.get_pk_constraint("Album"))

In [ ]:
for table_name in inspector.get_table_names():

    print(f"Table = {table_name}")
    
    print(f"Columns = ", end="")
    col_names_list = []
    for col in inspector.get_columns(table_name):
        col_names_list.append(col['name'])
    print(*col_names_list, sep=", ")
    
    print(f"Primary Keys = ", end="")
    pk_list = inspector.get_pk_constraint(table_name)
    pk_name_list = pk_list["constrained_columns"]
    print(*pk_name_list, sep=", ")

    fk_list = inspector.get_foreign_keys(table_name)
    if fk_list:
        print(f"Foreign Keys:")
        fk_name_list = []
        fk_reftbl_list = []
        fk_refcol_list = []
        
        for fk in fk_list:
            fk_name_list.append(*fk['constrained_columns'])
            fk_reftbl_list.append(fk['referred_table'])
            fk_refcol_list.append(*fk['referred_columns'])
            
        fk_info = zip(fk_name_list, fk_reftbl_list, fk_refcol_list)
        
        for n, t, c in fk_info:
            print(f"    {n} ---> {t}:{c}")

    print()

In [ ]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

Base = automap_base()
Base.prepare(autoload_with=engine)

print(*Base.metadata.tables, sep=", ")

In [ ]:
print(*Base.classes.keys(), sep=", ")

In [ ]:
def table_info(table_name):
    table = Base.metadata.tables[table_name]
    print(f"Table: {table_name}")
    print(f"Columns: ")
    for col in table.columns:
        print(f"   {col.name:15}", end="")
        if col.primary_key:
            print(f"*Primary Key*   ", end="")
        if col.foreign_keys:
            print(f"{col.foreign_keys}", end="")
        print()
    print()

In [ ]:
table_info("PlaylistTrack")
table_info("Track")

In [ ]:
Album = Base.classes.Album
Artist = Base.classes.Artist
Customer = Base.classes.Customer
Employee = Base.classes.Employee
Genre = Base.classes.Genre
Invoice = Base.classes.Invoice
InvoiceLine = Base.classes.InvoiceLine
MediaType = Base.classes.MediaType
Playlist = Base.classes.Playlist
Track = Base.classes.Track
playlisttrack = Base.metadata.tables['PlaylistTrack']

In [ ]:
import pandas as pd

#session1 = Session(engine)

albums = pd.read_sql_table(table_name='Album', con=engine)

print(albums)

In [ ]:
print(pd.options.display.max_rows)
print(pd.options.display.max_columns)

In [ ]:
artists = pd.read_sql_table(table_name='Artist', con=engine)

print(artists.head())

In [ ]:
df1 = pd.merge(left = albums, right = artists, how = 'inner')
print(df1.head())
print(df1.shape)

In [ ]:
df1.drop('ArtistId', axis=1).rename(columns = {'Name':'Artist'})

In [ ]:
df2 = (pd
     .merge(albums, artists)
     .drop('ArtistId', axis=1)
     .rename(columns = {'Name':'Artist'}))
print(df2.head())
print(df2.shape)

In [ ]:
tracks = pd.read_sql_table(table_name='Track', con=engine)

print(tracks.head())
print(tracks.shape)

In [ ]:
df3 = (pd
     .merge(df2, tracks)
     .drop(['AlbumId','TrackId',
            'Bytes','UnitPrice',
            'MediaTypeId','GenreId'], axis=1)
     .rename(columns = {'Name':'Track', 
                        'Title':'Album',
                        'Milliseconds':'Length(ms)'}))

print(df3.shape)
df3.head().style.format(thousands=",")

In [ ]:
print(f"Album table relationships")
print()
for relationship in inspect(Album).relationships:
    print(f"Relationship: {relationship}")
    print(f"Direction:    {relationship.direction}")
    print(f"Joined Table: {relationship.target}")
    print()

In [ ]:
from sqlalchemy import select

session = Session(engine)

statement = select(Album)
first_album = session.scalar(statement)

print(f"Album:  {first_album.Title}")
print(f"Artist: {first_album.artist.Name}")
print(f"Tracks:")
for t in first_album.track_collection:
    print(f"    {t.Name}")

session.close()

In [ ]:
print(f"Get inner join of tables Album, Track, and Artist")
statement = (select(Album, Track, Artist)
     .join(Track)
     .join(Artist)
    )

df4 = pd.read_sql(statement, con=engine)

print(df4.shape)
display(df4.head())

In [ ]:
print(f"Get inner join of tables Album, Track, and Artist")

statement = (select(Album.Title.label("Album"),
            Artist.Name.label("Artist"),
            Track.Name.label("Track"),
            Track.Composer, 
            Track.Milliseconds.label("Length"))
     .join(Track)
     .join(Artist)
    )

dataframe = pd.read_sql(sql=statement, con=engine)

print(dataframe.shape)
display(dataframe.head().style.format(thousands=","))

In [ ]:
albums = pd.read_sql_table(table_name='Album', con=engine)
artists = pd.read_sql_table(table_name='Artist', con=engine)
tracks = pd.read_sql_table(table_name='Track', con=engine)

df1 = (pd
     .merge(albums, artists)
     .rename(columns = {'Name':'Artist'}))

data = (pd
     .merge(df1, tracks)
     .rename(columns = {'Name':'Track', 
                        'Title':'Album',
                        'Milliseconds':'Length'}))

print(f"Longest track: {data.Length.max():d}")
print(f"Shortest track: {data.Length.min():,}")
print(f"How many blanks in Composer column?: "
      f"{data.Composer.isnull().sum():,}")
print(f"Track length mean: {data.Length.mean():,.2f}")
print(f"Track length median: {data.Length.median()}")
print(f"Artist mode: {data.Artist.mode()[0]}")
print(f"Correlation between Length and UnitPrice: "
      f"{data['Length'].corr(data['UnitPrice']):,.2f}")
print(f"Track length standard deviation: "
      f"{data.Length.std():,.2f}")

print(data.Artist.describe())


In [ ]:
from tabulate import tabulate

print(f"Track length standard deviation for a sample of artists:")
print(tabulate(
        data.groupby(['Artist'])['Length']
        .std()
        .dropna()
        .sample(3)
        .to_frame(), 
        headers = ['Artist','Track Length\nStd Dev'], 
        tablefmt="grid"))
print()
print(f"Longest tracks, with artist name:")
print(tabulate(
        data[['Track','Length','Artist']]
        .nlargest(3, 'Length'), 
        headers="keys", 
        tablefmt='grid', 
        showindex=False))
print()
print(f"Number of tracks per artist, from a sample of artists::")
print(tabulate(
        data
        .groupby('Artist')['Track']
        .count()
        .sample(3)
        .to_frame(), 
        headers = ['Artist','# Tracks'], 
        tablefmt="grid"))
print()
print(f"Shortest track by Artist=Guns N' Roses: ")
gnr = data.loc[data['Artist'] == "Guns N' Roses"]
gnr_shortest = gnr[['Track','Length']].nsmallest(3, 'Length')
print(tabulate(
        gnr_shortest, 
        headers="keys", 
        tablefmt='grid', 
        showindex=False))       

In [ ]:
print(f"Track length standard deviation for a sample of artists:")
pd.options.display.float_format = '{:,.2f}'.format
pd.options.styler.format.thousands= ','

display(
    data
    .groupby(['Artist'])['Length'] 
    .count()
    .sample(3)
    .to_frame()
)
print()
print(f"Longest tracks, with artist name:")
display(
    data[['Track','Length','Artist']]
    .nlargest(3, 'Length')
    .style.hide(axis="index")
)

print()
print(f"Number of tracks per artist, from a sample of artists::")
display(
    data
    .groupby('Artist')['Track']
    .count()
    .sample(3)
    .to_frame()
)
print()
print(f"Shortest tracks by Artist=Guns N' Roses: ")
gnr = data.loc[data.Artist == "Guns N' Roses"]
gnr_shortest = gnr[['Track','Length']].nsmallest(3, 'Length')
display(
    gnr_shortest
    .style.hide(axis="index")
)

In [ ]:
from sqlalchemy import select, func

with Session(engine) as session:
    statement = select(func.max(Track.Milliseconds))
    length = session.scalar(statement)

print(f"Longest track length: {length}")



In [ ]:
with Session(engine) as session:
    result = session.execute(statement)
    print(result)
    print(type(result))

In [ ]:
with Session(engine) as session:
    result = session.execute(statement)
    print(next(result))
    print(next(result))
    print(type(result))

In [ ]:
with Session(engine) as session:
    result = session.execute(statement)
    x = next(result).Artist
    print(x)
    print(type(x))

In [ ]:
with Session(engine) as session:
    limit = 4
    result = session.execute(statement)
    for index, item in enumerate(result, start=1):
        print(item)
        if index == limit:
            break

In [ ]:
table = []
limit = 4

with Session(engine) as session:
    result = session.execute(statement)
    for index, item in enumerate(result, start=1):
        table.append(item)
        if index == limit:
            break
    headers = session.execute(statement).keys()
    print(type(headers))
    
print(type(headers))
print(table)

In [ ]:
from collections import namedtuple

with Session(engine) as session:
    headers = session.execute(statement).keys()
    table = session.execute(statement).fetchmany(4)

print(type(headers))
print(next(iter(headers)))
print(type(table))
for row in table:
    print(type(row))

In [ ]:
from tabulate import tabulate

with Session(engine) as session:
    print(session.execute(statement))
    print()
    print(session.execute(statement).first())
    print()
    print(session.execute(statement).fetchmany(2))
    print()
    print(session.execute(statement.limit(2)).fetchall())
    print()
    print(session.scalars(statement))
    print()
    print(session.scalars(statement).first())
    print()
    print(session.scalars(statement).fetchmany(2))
    print()
    print(session.scalars(statement.limit(2)).fetchall())
    print()
    print(session.scalar(statement))

In [ ]:
statement = (select(Album.Title.label("Album"),
            Artist.Name.label("Artist"),
            Track.Name.label("Track"),
            Track.Composer, 
            Track.Milliseconds.label("Length"))
     .join(Track)
     .join(Artist)
     .where(Artist.Name == 'Alice In Chains')
    )

with Session(engine) as session:
    result = session.execute(statement).fetchall()
    
print()
for row in result:
    print(f"Track name: {row.Track:18} Composer: {row.Composer}")

In [ ]:
print(f"Experiment with returned data types\n")

statement = (select(Album.Title.label("Album"),
            Artist.Name.label("Artist"),
            Track.Name.label("Track"),
            Track.Composer, 
            Track.Milliseconds.label("Length"))
     .join(Track)
     .join(Artist)
    )

with Session(engine) as session:
    query = session.execute(statement.where(Artist.Name == 'Alice In Chains'))
    headings = query.keys()
    result = query.fetchmany(4)
    
#     query = session.execute(statement.where(Artist.Name == 'Alice In Chains').limit(4))
#     result_as_dicts = query.mappings().all()

print(result)

print()
for row in result:
    print(f"Track name: {row.Track}\tComposer: {row.Composer}")

print()
for row in result_as_dicts:
    print(row)

print()
for row in result:
    print(f"Track name: {row['Track']}\tComposer: {row['Composer']}")
    


In [ ]:
statement = (select(Artist).where(Artist.Name=='Alice In Chains'))

dataframe = pd.read_sql(sql=statement, con=engine)

print(dataframe.shape)
print(dataframe.head(5))

In [ ]:
print(f"Experiment with returned data types\n")

with Session(engine) as session:
    query = session.execute(statement.filter(Artist.Name == 'Alice In Chains'))
    headings = query.keys()
    result = query.fetchmany(4)
    
print(result)

print()
for row in result:
    print(f"Track name: {row.Track}\tComposer: {row.Composer}")

In [ ]:
with Session(engine) as session:
    query = session.execute(statement)
    result_headings = query.keys()
    result = query.fetchmany(5)

for x in result:
    print(x)

In [ ]:
print("similar to pandas select() method")
print()
with Session(engine) as session:
    result = session.execute(statement.order_by(func.random()).limit(2)).fetchall()

print(result)

In [ ]:
length = (session1
            .query(func.min(Track.Milliseconds))
            .scalar()
         )

print(f"Shortest track length: {length}")

In [ ]:
statement = (select(Artist))

dataframe = pd.read_sql(sql=statement, con=engine)

print(dataframe.shape)
print(dataframe.head(5))

In [ ]:
statement = (select(Artist).filter(Artist.Name=='Alice In Chains'))

dataframe = pd.read_sql(sql=statement, con=engine)

print(dataframe.shape)
print(dataframe.head(5))

In [ ]:
statement = (select(Artist).order_by(func.random()).limit(5))

dataframe = pd.read_sql(sql=statement, con=engine)

print(dataframe.shape)
print(dataframe.head(5))

In [ ]:
from sqlalchemy import select

session = Session(engine)

statement = select(Album)
first_album = session.scalars(statement.limit(3))
print(dir(first_album))

for x in first_album:

    print(f"Album:  {x.Title}")
    print(f"Artist: {x.artist.Name}")
    print(f"Tracks:")
    for t in x.track_collection:
        print(f"    {t.Name}")

session.close()

In [ ]:
statement = (select(Album.Title.label("Album"),
            Artist.Name.label("Artist"),
            Track.Name.label("Track"),
            Track.Composer, 
            Track.Milliseconds.label("Length"))
     .join(Track)
     .join(Artist)
    )

session = Session(engine)

first_album = session.execute(statement.limit(3))

for x in first_album:
    print(f"Album:  {x.Album}\tArtist: {x.Artist}\tTrack: {x.Track}")

print()

session.close()

In [ ]:
print(f"Names of all the tracks purchased by each customer")

statement = (select(Customer.FirstName,
                    Customer.LastName,
                    Customer.Country,
                    InvoiceLine.Quantity,
                    InvoiceLine.UnitPrice,
                    Track.Name,
                    Album.Title,
                    Artist.Name
                    )
                .join_from(Customer, Invoice)
                .join_from(Invoice, InvoiceLine)
                .join_from(InvoiceLine, Track)
                .join_from(Track, Album)
                .join_from(Album, Artist))

dataframe = pd.read_sql(sql=statement, con=engine)

print(dataframe.shape)
display(dataframe.head(5))




In [ ]:
# Creating ON statements where you need to be explicit about what is being joined

statement = (select(Customer.FirstName,
                    Customer.LastName,
                    Customer.Country,
                    InvoiceLine.Quantity,
                    InvoiceLine.UnitPrice,
                    Track.Name,
                    Album.Title,
                    Artist.Name,
                    Genre.Name
                    )
                .join(Invoice, Customer.CustomerId == Invoice.CustomerId)
                .join(InvoiceLine, Invoice.InvoiceId == InvoiceLine.InvoiceId)
                .join(Track, InvoiceLine.TrackId == Track.TrackId)
                .join(Album, Track.AlbumId == Album.AlbumId)
                .join(Genre, Track.GenreId == Genre.GenreId)
                .join(Artist, Album.ArtistId == Artist.ArtistId))

print(statement)
print()

statement = (select(Customer.FirstName,
                    Customer.LastName,
                    Customer.Country,
                    InvoiceLine.Quantity,
                    InvoiceLine.UnitPrice,
                    Track.Name,
                    Album.Title,
                    Artist.Name,
                    Genre.Name
                    )
                .join_from(Customer, Invoice)
                .join_from(Invoice, InvoiceLine)
                .join_from(InvoiceLine, Track)
                .join(Album)
                .join(Genre)
                .join(Artist))

print(statement)
print()

# statement = (select(Customer.FirstName,
#                     Customer.LastName,
#                     Customer.Country,
#                     InvoiceLine.Quantity,
#                     InvoiceLine.UnitPrice,
#                     Track.Name,
#                     Album.Title,
#                     Artist.Name,
#                     Genre.Name
#                     )
#                 .join_from(Artist, Album)
#                 .join_from(Genre, Track)
#                 .join_from(Album, Track)
#                  .join(InvoiceLine)
#                  .join(InvoiceLine)
#             )

session = Session(engine)
print(session.execute(statement).first())
print(session.execute(statement).fetchmany(2))
print(session.execute(statement.limit(2)).fetchall())
print(session.scalars(statement))
print(session.scalars(statement).first())
print(session.scalars(statement).fetchmany(2))
print(session.scalars(statement.limit(2)).fetchall())
print(session.scalar(statement))
session.close()

# join(user_table, address_table,
#          user_table.c.id == address_table.c.user_id)


dataframe = pd.read_sql(sql=statement, con=engine)
print()
print(dataframe.shape)
display(dataframe.head(5))

In [ ]:
print(f"Names of all the tracks purchased by each customer")

statement = (select(Customer.FirstName,
                    Customer.LastName,
                    Customer.Country,
                    InvoiceLine.Quantity,
                    InvoiceLine.UnitPrice
                    )
                .select_from(Customer)
                .join(Invoice)
                .select_from(Invoice)
                .join(InvoiceLine))

dataframe = pd.read_sql(sql=statement, con=engine)

print(dataframe.shape)
display(dataframe.head(5))


In [ ]:
print(f"Names of all the tracks purchased by each customer")

statement = (select(Customer.FirstName,
                    Customer.LastName,
                    Customer.Country,
                    Track.Name.label("Track"),
                    Album.Title.label("Album"),
                    Artist.Name.label("Artist"),
                    InvoiceLine.Quantity,
                    InvoiceLine.UnitPrice
                    )
                .join(InvoiceLine)
                .join(Customer, Invoice.CustomerId)
                .join_from(InvoiceLine, Track)
                .join_from(Track, Album)
                .join_from(Album, Artist))

print(statement)
# dataframe = pd.read_sql(sql=statement, con=engine)

# print(dataframe.shape)
# display(dataframe.head(5))

In [ ]:
print(f"Names of all the tracks purchased by each customer")

statement = (select(Playlist.Name.label("Playlist"),
                    Track.Name.label("Track"),
                    Album.Title.label("Album"),
                    Artist.Name.label("Artist")
                    )
                .join_from(Playlist, playlisttrack)
                .join_from(playlisttrack, Track)
                .join_from(Track, Album)
                .join_from(Album, Artist))

dataframe = pd.read_sql(sql=statement, con=engine)

print(dataframe.shape)
display(dataframe.head(5).style.format(thousands=","))

In [ ]:
statement = (select(Playlist.Name.label("Playlist"),
                    Track.Name.label("Track"),
                    Album.Title.label("Album"),
                    Artist.Name.label("Artist")
                    )
                .join_from(Playlist, playlisttrack)
                .join_from(playlisttrack, Track)
                .join_from(Track, Album)
                .join_from(Album, Artist))

dataframe = pd.read_sql(sql=statement.order_by(func.random()).limit(5), con=engine)

print(dataframe.shape)
display(dataframe.head(5))

In [ ]:
from sqlalchemy import create_engine, inspect, select
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import pandas as pd
from tabulate import tabulate

engine = create_engine(r"sqlite:///C:/Users/blinklet/Documents/chinook-database/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite")
Base = automap_base()
Base.prepare(autoload_with=engine)

Album = Base.classes.Album
Artist = Base.classes.Artist
Customer = Base.classes.Customer
Employee = Base.classes.Employee
Genre = Base.classes.Genre
Invoice = Base.classes.Invoice
InvoiceLine = Base.classes.InvoiceLine
MediaType = Base.classes.MediaType
Playlist = Base.classes.Playlist
Track = Base.classes.Track
playlisttrack = Base.metadata.tables['PlaylistTrack']

statement = (select(Playlist.Name.label("Playlist"),
                Track.Name.label("Track"),
                Album.Title.label("Album"),
                Artist.Name.label("Artist")
                )
            .join_from(Playlist, playlisttrack)
            .join_from(playlisttrack, Track)
            .join_from(Track, Album)
            .join_from(Album, Artist))

with Session(engine) as session4:
    headers = session.execute(statement).keys()
    table = session.execute(statement).fetchmany(4)

print(tabulate(table, headers, tablefmt='grid'))

In [1]:
from sqlalchemy import create_engine, select, func
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

engine = create_engine(r"sqlite:///C:/Users/blinklet/Documents/chinook-database/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite")

Base = automap_base()
Base.prepare(autoload_with=engine)

Album = Base.classes.Album
Artist = Base.classes.Artist
Customer = Base.classes.Customer
Employee = Base.classes.Employee
Genre = Base.classes.Genre
Invoice = Base.classes.Invoice
InvoiceLine = Base.classes.InvoiceLine
MediaType = Base.classes.MediaType
Playlist = Base.classes.Playlist
Track = Base.classes.Track
playlisttrack = Base.metadata.tables['PlaylistTrack']



In [37]:
with Session(engine) as session:
    statement = select(func.max(Track.Milliseconds))
    length = session.scalar(statement)
    print(f"Longest track length: {length}")

    statement = select(func.min(Track.Milliseconds))
    length = session.scalar(statement)
    print(f"Shortest track length: {length}")
    
    statement = select(func.count(Track.Composer))
    statement2 = select(func.count(Track.TrackId))
    composer_rows = session.scalar(statement)
    all_rows = session.scalar(statement2)
    empty_rows = all_rows - composer_rows
    print(f"How many blanks in Composer column?: {x}")

    statement = select().where(Track.Composer == None)
    composer_rows = session.execute(statement)
    print("T",composer_rows)
    #query.filter(User.name.isnot(None))

#     s = book_publisher.select().where(
#     book_publisher.c.publisherName == None)
    
    statement = select(func.avg(Track.Milliseconds))
    mean = session.scalar(statement)
    print(f"Mean track length: {mean:,.0f}")


    

Longest track length: 5286953
Shortest track length: 1071
How many blanks in Composer column?: 978


OperationalError: (sqlite3.OperationalError) near "FROM": syntax error
[SQL: SELECT  
FROM "Track" 
WHERE "Track"."Composer" IS NULL]
(Background on this error at: https://sqlalche.me/e/14/e3q8)